# 2-round 3-branch Griffin Optimal Modeling
### initialization

In [1]:
from sage.rings.polynomial.polynomial_gf2x import GF2X_BuildIrred_list

prime = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001

Fp = GF(prime)

t = 3
d = 5
d_inv = d.inverse_mod(prime - 1)
round_constants = [
    [
        Fp(0x0738f78efd46ac801667362968f2ed3e93a588629f0a8067af537055ef3b8de5), 
        Fp(0x08606cb2fdb16bc68da0ac1809d90f0f3ed1c8cb6ffb202ae68e0483dab296eb), 
        Fp(0x3bbf8bece610e71898268fa9e82c81c53c8a13d0e5dfe0eac3dc41c01b43c56e)
    ], 
    [
        Fp(0x32df86c26e4cba04adc1eaf7d87ecafe35760975fcf80980598d927fceb2fc8e), 
        Fp(0x153d2fa172d4bc90d53fd5d6f198f5e8b14632a8388d9c76be162b4e9992b833), 
        Fp(0x3ef2caf6b3356646639487a4eef208fcba88b366518fccd3cbbdd115b324d450)
    ], 
    [
        Fp(0x4df9667b63073058c6de8747008bdc5fc916180c560dc2bd68a1ef5fa28ffc6e), 
        Fp(0x576a9cbd19be1ba96436f2836af9855527b0e974cd119049ed418d4a85486945), 
        Fp(0x2a609446e01675490a9b3c51f13ee5794a32c6f3d81ce9c9a5897156d7348033)
    ], 
    [
        Fp(0x41b1d71f98a56c8edfb28f3aa869c23c41ff2b5705c54e6958506dd414b22459), 
        Fp(0x2c19713b52a1ae3565789a59051530429a26714eb1433e75f5cf6ab9ec71a9d1), 
        Fp(0x461e6c2ad5ea136098a6a15d012bed4da101c302b588fc2949fd14c99ccae765)
    ], 
    [
        Fp(0x2ac8dbbb2704463c04245bf11a272e1fb2e6d283786a23314fc7d27ec1bb10e0), 
        Fp(0x34c930ef1931d6571eb3d1a37d699aeed1da7209f7366f6fd218235aa8ef3743), 
        Fp(0x480523d6a067ce2a2bee34c7f7e3ff839fb621ba030fb2072f3a8e66b3f3a437)
    ], 
    [
        Fp(0x69dfffdedfef891b448a1adee96de4d8828ac44d39803bc7ac55b62acf948e23), 
        Fp(0x307f367156f88b9f63ae791d3de97ea7f56152107fcd448ef9741b0d9c6bad19), 
        Fp(0x0eb8c7dd9af6474636fd3970460a24e5ba90cd0c5b9fd14991791a4610763e51)
    ], 
    [
        Fp(0x0886bf20a448bd83746fe5e21995cf73911701f7ee2761575d47c44b3827dde5), 
        Fp(0x4f1b3f6a9f91cfc548389964eefe28a3a9572224c5541962011b540f60b509d8), 
        Fp(0x2e516b00cf8f54ff284c394d51f863d3bbdb95e7271c46f5281e3e3bbf55dd6a)
    ], 
    [
        Fp(0x0833e4089451bfdbf35aeed2b89b11023954a4b43567ab4b1fac69756736b33a), 
        Fp(0x42f66b34799969b80ff2eeeb9e597fe80103073a3097cfda7cb422246215f5c4), 
        Fp(0x0110a55febe4caff6421338acafff4b8a56703ab30819a536337504a443180ae)
    ], 
    [
        Fp(0x1315433a9dc7e343b42964b26ca6b36c1617c66c508553e123f10257ddd3600e), 
        Fp(0x1991e3e8fc0aaf0b41f7ff13eb4d730f19555e61a8d1a7af2399676fca434c21), 
        Fp(0x4227975a789e034acc7a0e261266aee63cc0a40cbdaddf58cdc2ba4efb1cb1df)
    ], 
    [
        Fp(0x286cf3079a8c91fb8bd8c3a02371541cb52de94e6ea228cb4c7491cd325f657b), 
        Fp(0x101af3fa3e29010d7ae82ce1635a0c0f625f7a05ddad839d014c3d002f2278ad), 
        Fp(0x399f684104e07565ac718896cfae286f60acc1037f900bb3038fa0dbc6eaa2b9)
    ], 
    [
        Fp(0x31f3c4fef008e85cc4b821a0a32aa30de0a8b452106bd041cb313406c1ca5156), 
        Fp(0x072d18023e12c43653ebc99bee5de3d563fbc9351a4286d89998be9a279b65c0), 
        Fp(0x3871d46d523094488e30427b556551187f803dc599dc6dcb67121c1fce725be6)
    ]
]
alpha2 = Fp(0x3b97b6663ce81ef95292208190f07a44178da81b1d6bf3c8699b7f878a844909)
beta2 = Fp(0x102da31073257dda8f45d2d74903cff1d2e96b7f654af6c481b119fa677ef06b)

M3 = matrix(Fp, 3, 3, [
    [2, 1, 1], 
    [1, 2, 1], 
    [1, 1, 2]
])
M3_inv = M3 ^ -1

def mul_M(state):
    return list(M3 * vector(state))

def G2(y0, y1):
    return (y0 + y1)^2 + alpha2 * (y0 + y1) + beta2

def S(state):
    y0 = state[0]^d_inv
    y1 = state[1]^d
    y2 = state[2] * G2(y0, y1)
    return [y0, y1, y2]

def round_function(state, r):
    state = S(state)
    state = mul_M(state)
    for i in range(t):
        state[i] += round_constants[r][i]
    return state

def Griffin_pi(state, rounds):
    state = mul_M(state)
    for r in range(rounds - 1):
        state = round_function(state, r)
    state = S(state)
    state = mul_M(state)
    return state

def print_hex(state):
    for x in state:
        print(hex(x))

### 12-round test vectors

In [2]:
# random message
p0 = Fp(0x3c5afec39c59e6e79f8c91937d926e73522aa753d36ba0dafeaeb32c644832fc)
p1 = Fp(0x600b8ffd1c79a6cdebf88e16650a1246ad6ad84ef23958f6c547f2e019c34abc)
p2 = Fp(0x00af42d52bae842a0b28abb13b65e710421309e80985fb47e9bef91c37f2856a)
state = Griffin_pi([p0, p1, p2], 12)
print("***************************")
print("messages：")
print_hex([p0, p1, p2])
print("summaries：")
print_hex(state)

# All 0 News
state = Griffin_pi([0, 0, 0], 12)
print("***************************")
print("messages：")
print_hex([0, 0, 0])
print("summaries：")
print_hex(state)

***************************
messages：
0x3c5afec39c59e6e79f8c91937d926e73522aa753d36ba0dafeaeb32c644832fc
0x600b8ffd1c79a6cdebf88e16650a1246ad6ad84ef23958f6c547f2e019c34abc
0xaf42d52bae842a0b28abb13b65e710421309e80985fb47e9bef91c37f2856a
summaries：
0x4aadb652f349260450d733db55c048af3c0ec47cf7d6ab3787e9096a6ab408ec
0xf9291eae795d6260ae4a8ed20eb68e71fa55fd1677bf623e2baf8f5fd58f91d
0x275e1300e5b81cda8316b6c1ad8a5addb5a462f592a803e0fadbb8ea353e2c52
***************************
messages：
0x0
0x0
0x0
summaries：
0x4b68e0cb35cca218e75b9f0df92eaa6dd0ab1465bd4f91340544735ef73bc242
0x1b7f6fcc56baf31690f3a7b3bc115445cd14cab7652ac7e4eac36552850619b7
0xc80845e90a5377cbc2ec8c631f3dedf010b86a434beaf0b96eae4025f549f69


### 2-round 3-branch Griffin Optimal Modeling System of Equations

In [3]:
R.<x1, x2, x3> = PolynomialRing(Fp)
############
a00 = x1
b00 = Fp(0)
r00 = Fp(0)
[a01, b01, r01] = list(M3 * vector([a00, b00, r00]))
############
b10 = b01^d
a11 = x3^d
b11 = x2
r11 = (b10 - M3_inv[1][0] * (a11 - round_constants[0][0]) - M3_inv[1][1] * (b11 - round_constants[0][1])) / M3_inv[1][2] + round_constants[0][2]
[a10, tmp, r10] = list(M3_inv * vector([a11 - round_constants[0][0], b11 - round_constants[0][1], r11 - round_constants[0][2]]))
assert(b10 == tmp)
g1 = G2(a10, b10)
############
a20 = x3
b20 = b11^d
g2 = G2(a20, b20)
r20 = r11 * g2

In [6]:
def print_magma(Eqs, Vlist, magma_file):
    f = open(magma_file, 'w')
    f.write("Fp := GF({});\n".format(prime))
    f.write("R<" + ", ".join(Vlist) + "> := PolynomialRing(Fp, {}, \"lex\");\n".format(len(Vlist)))

    s = ""
    for i in range(len(Eqs)-1):
        s += "f{}, ".format(i)
    s += "f{}".format(len(Eqs)-1)    
    for i in range(len(Eqs)):        
        f.write("f{} := ".format(i) + str(Eqs[i]) + ";\n")

    f.write("I := ideal<R|" + s + ">;\n")
    f.write("time gb := GroebnerBasis(I : Al := \"FGLM\");\n")
    f.write("Variety(I);\n")
    f.close()

In [5]:
# Ask for the original image.
equs = []
equs.append(a10^d - a01)
equs.append(r01 * g1 - r10)
equs.append(M3[0][0] * a20 + M3[0][1] * b20 + M3[0][2] * r20 - Fp(0x4aadb652f349260450d733db55c048af3c0ec47cf7d6ab3787e9096a6ab408ec))
print_magma(equs, ["x1", "x2", "x3"], "2r_3t.mag")

In [6]:
# Verification of the original image
p0 = Fp(12092689199522945160224628114827463362977434653644987287917624730202928336012)
p1 = Fp(0)
p2 = Fp(0)
state = Griffin_pi([p0, p1, p2], 2)
print("***************************")
print("messages：")
print_hex([p0, p1, p2])
print("summaries：")
print_hex(state)
############
p0 = Fp(34290167009979146033423565554198793766443289974142583767619719918922631927197)
state = Griffin_pi([p0, p1, p2], 2)
print("***************************")
print("messages：")
print_hex([p0, p1, p2])
print("summaries：")
print_hex(state)

***************************
messages：
0x1abc3833927ceac95a5d02566772af2b46fb1fbfa704fd45637c3f170b944c8c
0x0
0x0
summaries：
0x4aadb652f349260450d733db55c048af3c0ec47cf7d6ab3787e9096a6ab408ec
0x325ec0789303b0a9e9062f0794fcffad35eb0312f2ebaf377d280858976b41ec
0x37519e24a2302693f66f139c3626c9a0a5c077f018d414333dd832838dd134f5
***************************
messages：
0x4bcf8bf7d0acb9047880c93d86504a286a9f4f5a69f655d7499df4d9b9bad59d
0x0
0x0
summaries：
0x4aadb652f349260450d733db55c048af3c0ec47cf7d6ab3787e9096a6ab408ec
0x5d7513e18981e6f440a6249b23a54c9325f99adf474d5bf8de1e5c1bb4d46329
0x3ea246b799f4c02054c36bfb09f672825e6be5a92aa927e1fc9bb3c274a3b342


### 2 Wheel 3 Branch 1 Degree of Freedom Griffin Optimal Modeling System of Equations

In [36]:
R.<x1, x2> = PolynomialRing(Fp)
############ Round 2
const = Fp(0)
a21 = Fp(0x4aadb652f349260450d733db55c048af3c0ec47cf7d6ab3787e9096a6ab408ec)
b20 = const
r20 = x2
a20 = (a21 - b20 - r20) / 2
g2 = G2(a20, b20)
############ Round 1
a11 = a20 ^ d
b11 = b20 ^ d_inv
b10 = x1 ^ d
r11 = (b10 - M3_inv[1][0] * (a11 - round_constants[0][0]) - M3_inv[1][1] * (b11 - round_constants[0][1])) / M3_inv[1][2] + round_constants[0][2]
[a10, tmp, r10] = list(M3_inv * vector([a11 - round_constants[0][0], b11 - round_constants[0][1], r11 - round_constants[0][2]]))
assert(b10 == tmp)
g1 = G2(a10, b10)
############ Round 0
a01 = a10 ^ d
b01 = x1
r01 = (0 - M3_inv[2][0] * a01 - M3_inv[2][1] * b01) / M3_inv[2][2]
[a00, b00, r00] = list(M3_inv * vector([a01, b01, r01]))
assert(r00 == 0)
############ Ask for the original image.
equs = []
equs.append(r01 * g1 - r10)
equs.append(r11 * g2 - r20)
print_magma(equs, ["x1", "x2"], "2r_3t_1f.mag")

In [38]:
a = Fp(8886967542346947864499000035344828781312230968998272027128458638772857403403)
b = Fp(2797945543165789615374065870189492538986431060789023429416710367902128947345)
state = [a01(a, b), b01(a, b), r01(a, b)]
state = list(M3_inv * vector(state))
print("***************************")
print("messages：")
print_hex(state)
state = Griffin_pi(state, 2)
print("summaries：")
print_hex(state)

***************************
messages：
0x168b97fe481274438208c103746b91ef24788d8c939a57b6dc198ca53a62cf0e
0x3883f3dfc9222d7a9ceb14308b2d42b691f11273479d3a9650c3e69f1170147f
0x0
summaries：
0x4aadb652f349260450d733db55c048af3c0ec47cf7d6ab3787e9096a6ab408ec
0x62657933257d3b9d93a33e6afcd11d7849e53211e96d2b2ce8f0241a704824bf
0x68950df353910f8c36d8af5d971137b44de32db5c4727a5032e762e5e6246550


### 2 Wheel 3 Branch 1 Degree of Freedom Griffin Optimal Modeling System of Equations (S box count of 3)

In [7]:
R.<x1, x2> = PolynomialRing(Fp)
d = 3
############ Round 1
a11 = x2^d
b11 = 0
r11 = (x1^d - M3_inv[1][0] * (x2^d - round_constants[0][0]) - M3_inv[1][1] * (b11 - round_constants[0][1])) / M3_inv[1][2] + round_constants[0][2]
[a10, b10, r10] = list(M3_inv * vector([a11 - round_constants[0][0], b11 - round_constants[0][1], r11 - round_constants[0][2]]))
assert(b10 == x1^d)
############ Round 2
a20 = x2
b20 = b11^d
r20 = r11 * G2(a20, b20)
a21 = M3[0][0] * a20 + M3[0][1] * b20 + M3[0][2] * r20
############ Round 0
a01 = a10^d
b01 = x1
r01 = -(M3_inv[2][0] * a01 + M3_inv[2][1] * b01) / M3_inv[2][2]
############ Ask for the original image.
equs = []
equs.append(a21)
equs.append(r01 * G2(a10, b10) - r10)
print_magma(equs, ["x1", "x2"], "2r_3t_1f_3d.mag")